In [1]:
import sys
import os
import bungee_py

In [3]:
import yaml
import pint

UREG = pint.UnitRegistry()
# UREG.setup_matplotlib(True)

path = os.path.join(os.environ["SRC_DIR"], "examples", "115ft_30min.yaml")

with open(path, "r") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)

# water type
water = getattr(bungee_py.Water, data["water"])

# SCR
working_scr_pint = UREG.parse_expression(data["scr"]["work"])
deco_scr_pint = UREG.parse_expression(data["scr"]["deco"])
# scr = bungee_py.Scr(
#     work = ,
#     deco = 
# )
#  = 
# default_scr = Scr(default_volume_rate)

# # tank info
# tank_info = {}
# for entry in data["tanks"]:
#     name = entry["name"]
#     enum = Tank[entry["type"]]
#     pressure = UREG.parse_expression(entry["pressure"])
#     mix = Mix(po2=entry["mix"]["po2"])
#     tank_info[name] = TankInfo(enum, pressure, mix)

# # decompression
# deco = BuhlmannParams(gf_low=data["deco"]["gf_low"], gf_high=data["deco"]["gf_high"])

# # create the plan object
# plan = Plan(water, default_scr, tank_info, deco)

# ## profile
# # get a list of times in minutes
# durations = [
#     UREG.parse_expression(p["duration"]).to(config.TIME_UNIT).magnitude for p in data["profile"]
# ]
# times = [float(d) * config.TIME_UNIT for d in np.cumsum([0.0] + durations)]
# # get a list of depths
# depths = [0 * config.DEPTH_UNIT] + [UREG.parse_expression(p["depth"]) for p in data["profile"]]
# # scr points
# scrs = [
#     Scr(UREG.parse_expression(p["scr"])) if "scr" in p else None for p in data["profile"]
# ] + [None]
# # tank points
# tanks = [p["tank"] if "tank" in p else None for p in data["profile"]]
# tanks += [tanks[-1]]
# if tanks[0] is None:
#     raise Exception("First entry in profile must contain tank")

# N = len(times)

# # other checks
# assert len(depths) == N
# assert len(scrs) == N
# assert len(tanks) == N

# # populate the profile points
# for i in range(N):
#     plan.add_point(time=times[i], depth=depths[i], scr=scrs[i], tank_name=tanks[i])

# plan.validate()

# return plan